In [1]:
import pandas as pd
import requests

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1763436619,2025-11-18 00:30,1763436600,2025-11-18 00:30,15.1,59.2,0,Encoberto,//cdn.weatherapi.com/weather/64x64/night/122.png,1009,5.4,8.6,80,E,1014.0,29.94,0.57,0.02,100,100,15.1,59.2,14.1,57.4,14.4,58.0,14.1,57.5,10.0,6.0,0.0,7.6,12.2,"[{'date': '2025-11-18', 'date_epoch': 17634240..."


In [4]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-11-18,1763424000,"[{'time_epoch': 1763434800, 'time': '2025-11-1...",19.1,66.4,12.7,54.8,15.3,59.5,10.5,16.9,19.74,0.78,0.0,7.3,4.0,92,1,93,0,0,Chuva moderada,//cdn.weatherapi.com/weather/64x64/day/302.png,1189,1.2,05:20 AM,06:45 PM,03:59 AM,05:32 PM,Waning Crescent,5,0,0


In [5]:
# Extrair a lista de horas do primeiro dia de previsão
hour_list = df_forecast['hour'][0]  # Pega a lista de horas do primeiro dia

# Agora normalize essa lista para criar o dataframe de horas
df_hour = pd.json_normalize(hour_list)

# Converter para datetime
df_hour["time"] = pd.to_datetime(df_hour["time"])

# Definir como índice
df_hour = df_hour.set_index("time")

# Selecionar apenas colunas numéricas
df_numeric = df_hour.select_dtypes(include="number")

# Reamostrar de 5h em 5h e calcular média
df_5h = df_numeric.resample("5h").mean().reset_index()

print(df_5h.head())
df_5h

                 time    time_epoch  temp_c  temp_f  is_day  wind_mph  \
0 2025-11-18 00:00:00  1.763442e+09  14.500   58.14     0.0      5.64   
1 2025-11-18 05:00:00  1.763460e+09  14.860   58.78     0.8      3.02   
2 2025-11-18 10:00:00  1.763478e+09  17.580   63.66     1.0      5.66   
3 2025-11-18 15:00:00  1.763496e+09  16.220   61.18     0.8      9.14   
4 2025-11-18 20:00:00  1.763512e+09  12.875   55.20     0.0      7.35   

   wind_kph  wind_degree  pressure_mb  pressure_in  precip_mm  precip_in  \
0      9.06         75.2      1010.80      29.8480      0.578      0.022   
1      4.82         62.6      1009.60      29.8140      2.818      0.112   
2      9.06        254.0      1007.00      29.7360      0.326      0.014   
3     14.68        281.6      1007.00      29.7320      0.218      0.008   
4     11.80        265.5      1009.25      29.8025      0.015      0.000   

   snow_cm  humidity  cloud  feelslike_c  feelslike_f  windchill_c  \
0      0.0      98.6  100.0       

,time,time_epoch,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition.code
0,2025-11-18 00:00:00,1.763442e+09,14.500,58.14,0.0,5.64,9.06,75.2,1010.80,29.8480,0.578,0.022,0.0,98.6,100.0,14.02,57.220,14.02,57.220,14.36,57.90,14.12,57.40,0.80,80.0,0.0,0.0,4.2,2.4,8.100,13.06,0.00,1123.0
1,2025-11-18 05:00:00,1.763460e+09,14.860,58.78,0.8,3.02,4.82,62.6,1009.60,29.8140,2.818,0.112,0.0,98.8,100.0,15.30,59.480,15.30,59.480,14.88,58.78,14.66,58.40,1.00,100.0,0.0,0.0,6.6,3.8,4.360,7.00,0.10,1192.0
2,2025-11-18 10:00:00,1.763478e+09,17.580,63.66,1.0,5.66,9.06,254.0,1007.00,29.7360,0.326,0.014,0.0,93.8,88.4,17.60,63.660,17.60,63.660,17.60,63.66,16.54,61.74,1.00,97.0,0.0,0.0,8.0,4.8,7.740,12.44,3.18,1225.6
3,2025-11-18 15:00:00,1.763496e+09,16.220,61.18,0.8,9.14,14.68,281.6,1007.00,29.7320,0.218,0.008,0.0,82.2,65.8,15.84,60.500,15.84,60.500,16.22,61.18,13.20,55.74,0.80,76.4,0.0,0.0,9.6,5.6,13.960,22.48,2.40,1050.4
4,2025-11-18 20:00:00,1.763512e+09,12.875,55.20,0.0,7.35,11.80,265.5,1009.25,29.8025,0.015,0.000,0.0,88.0,51.5,11.85,53.325,11.85,53.325,12.90,55.20,10.95,51.75,0.25,34.0,0.0,0.0,10.0,6.0,12.225,19.65,0.00,1033.0


In [6]:
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
]

df_limpo = df_5h[[c for c in colunas_importantes if c in df_5h.columns]]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain
1,2025-11-18 05:00:00,15.30,98.8,100.0,0.10,4.82,1.00,100.0
0,2025-11-18 00:00:00,14.02,98.6,100.0,0.00,9.06,0.80,80.0
3,2025-11-18 15:00:00,15.84,82.2,65.8,2.40,14.68,0.80,76.4
2,2025-11-18 10:00:00,17.60,93.8,88.4,3.18,9.06,1.00,97.0
4,2025-11-18 20:00:00,11.85,88.0,51.5,0.00,11.80,0.25,34.0


In [7]:
import os
print(os.getcwd())

c:\Users\diona\OneDrive\Área de Trabalho\wardobre_ipynb\BACKEND
